In [47]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import tensorflow as tf

In [62]:
classifier = Sequential()

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = 'relu', input_shape = (28, 28,1)))
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))
               
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))
               
classifier.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))


classifier.add(Flatten())
               
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 50, activation = 'softmax'))

opt=tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)              
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [63]:
for layer in classifier.layers:
    print(layer.output_shape)

(None, 28, 28, 64)
(None, 28, 28, 64)
(None, 14, 14, 64)
(None, 14, 14, 64)
(None, 14, 14, 128)
(None, 14, 14, 128)
(None, 7, 7, 128)
(None, 7, 7, 128)
(None, 7, 7, 256)
(None, 7, 7, 256)
(None, 3, 3, 256)
(None, 3, 3, 256)
(None, 2304)
(None, 128)
(None, 50)


In [10]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.Collecting split-folders



You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [11]:
import splitfolders

In [12]:
splitfolders.ratio("DataSet/Train", output="DataSet/main", seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

In [64]:
train_folder = "Dataset/main/train"
val_folder = "Dataset/main/val"
test_folder = "Dataset/test"

In [65]:
Image.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(shear_range = .2, rotation_range = 25)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_set = train_datagen.flow_from_directory(train_folder, target_size = (28, 28), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")
val_set = val_datagen.flow_from_directory(val_folder, target_size = (28, 28), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory(test_folder, target_size = (28, 28), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")

Found 10800 images belonging to 50 classes.
Found 1200 images belonging to 50 classes.
Found 3000 images belonging to 50 classes.


In [52]:
classifier.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_45 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 dropout_21 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 conv2d_46 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_47 (Conv2D)          (None, 14, 14, 128)       147584    
                                                      

In [66]:
classifier.fit(train_set, steps_per_epoch = 10800, epochs = 10,
                        validation_data = val_set, validation_steps = 1200)

Epoch 1/10
10800/10800 [==============================] - 446s 41ms/step - loss: 3.6813 - accuracy: 0.0710 - val_loss: 2.8765 - val_accuracy: 0.2242
Epoch 2/10
10800/10800 [==============================] - 371s 34ms/step - loss: 2.6668 - accuracy: 0.2669 - val_loss: 1.7326 - val_accuracy: 0.4825
Epoch 3/10
10800/10800 [==============================] - 468s 43ms/step - loss: 2.1512 - accuracy: 0.3919 - val_loss: 1.3057 - val_accuracy: 0.6083
Epoch 4/10
10800/10800 [==============================] - 443s 41ms/step - loss: 1.8537 - accuracy: 0.4836 - val_loss: 1.1248 - val_accuracy: 0.6642
Epoch 5/10
10800/10800 [==============================] - 399s 37ms/step - loss: 1.7579 - accuracy: 0.5210 - val_loss: 1.2421 - val_accuracy: 0.6375
Epoch 6/10
10800/10800 [==============================] - 351s 33ms/step - loss: 1.7979 - accuracy: 0.5207 - val_loss: 1.0771 - val_accuracy: 0.7092
Epoch 7/10
10800/10800 [==============================] - 400s 37ms/step - loss: 1.8623 - accuracy: 0.5181

In [67]:
classifier.evaluate(test_set)

3000/3000 [==============================] - 27s 9ms/step - loss: 1.2478 - accuracy: 0.6523


[1.2477961778640747, 0.6523333191871643]